In [1]:
# 2 挖掘数据中的关联规则

In [121]:
## Apriori

In [122]:
from efficient_apriori import apriori
# 设置数据集
transactions = [('牛奶','面包','尿布'),
		('可乐','面包', '尿布', '啤酒'),
		('牛奶','尿布', '啤酒', '鸡蛋'),
		('面包', '牛奶', '尿布', '啤酒'),
		('面包', '牛奶', '尿布', '可乐')]
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(transactions, min_support=0.5,  min_confidence=1)
print("频繁项集：", itemsets)
print("关联规则：", rules)

频繁项集： {1: {('面包',): 4, ('牛奶',): 4, ('尿布',): 5, ('啤酒',): 3}, 2: {('尿布', '牛奶'): 4, ('尿布', '面包'): 4, ('牛奶', '面包'): 3, ('啤酒', '尿布'): 3}, 3: {('尿布', '牛奶', '面包'): 3}}
关联规则： [{牛奶} -> {尿布}, {面包} -> {尿布}, {啤酒} -> {尿布}, {牛奶, 面包} -> {尿布}]


In [107]:
## Regression

In [108]:
### Pearson

In [109]:
import pandas as pd

def compute(x):
    return 2*x*x+1
x=[i for i in range(100)]
y=[compute(i) for i in x]
data = pd.DataFrame({'x':x,'y':y})
# 查看pearson系数
print(data.corr())
print(data.corr(method='spearman'))
print(data.corr(method='kendall'))


          x         y
x  1.000000  0.967644
y  0.967644  1.000000
     x    y
x  1.0  1.0
y  1.0  1.0
     x    y
x  1.0  1.0
y  1.0  1.0


In [110]:
### 回归分析
import random
from sklearn import linear_model

In [111]:
reg = linear_model.LinearRegression()

In [112]:
def generate(x):
	y = 2*x+10+random.random()
	return y

train_x = []
train_y = []

In [113]:
for x in range(1000):
	train_x.append([x])
	y = generate(x)
	train_y.append([y])

reg.fit (train_x, train_y)

LinearRegression()

In [114]:
# coef_ 保存线性模型的系数w
print(reg.coef_)
print(reg.intercept_)

[[1.99996812]]
[10.50946744]


### 使用sklearn自带的糖尿病数据集，进行回归分析

Diabetes：包含442个患者的10个生理特征（年龄，性别、体重、血压）和一年以后疾病级数指标

In [115]:
from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split  
from sklearn.metrics import mean_squared_error

In [116]:
# 加载数据
diabetes = datasets.load_diabetes()
data = diabetes.data
#import numpy as np
#print(np.max(data))
#data.to_csv('diabetes.csv')

In [117]:
# 数据探索
print(data.shape)
print(data[0])

(442, 10)
[ 0.03807591  0.05068012  0.06169621  0.02187235 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990842 -0.01764613]


In [118]:
# 训练集 70%，测试集30%
train_x, test_x, train_y, test_y = train_test_split(diabetes.data, diabetes.target, test_size=0.3, random_state=14)
print(len(train_x))

309


In [119]:
#回归训练及预测
clf = linear_model.LinearRegression()
clf.fit(train_x, train_y)

LinearRegression()

In [120]:
print(clf.coef_)
#print(train_x.shape)
#print(clf.score(test_x, test_y))
pred_y = clf.predict(test_x)
print(mean_squared_error(test_y, pred_y))
r_sq = clf.score(train_x, train_y) #确定系数
print('r_sq:', r_sq)


[  32.03000032 -228.38626681  492.80665731  313.61844116 -991.31389923
  551.99413533  190.16297006  278.51146815  781.03825662   72.08348977]
3180.3670319563707
r_sq: 0.5194074106259234


In [2]:
## BreadBasket

In [3]:
import pandas as pd
import time

In [4]:
# 数据加载
data = pd.read_csv('~/data/Issue_004/BreadBasket_DMS.csv')
# 统一小写
data['Item'] = data['Item'].str.lower()
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)

In [5]:
# 采用efficient_apriori工具包
def rule1():
	from efficient_apriori import apriori
	start = time.time()
	# 得到一维数组orders_series，并且将Transaction作为index, value为Item取值
	orders_series = data.set_index('Transaction')['Item']
	# 将数据集进行格式转换
	transactions = []
	temp_index = 0
	for i, v in orders_series.items():
		if i != temp_index:
			temp_set = set()
			temp_index = i
			temp_set.add(v)
			transactions.append(temp_set)
		else:
			temp_set.add(v)
	
	# 挖掘频繁项集和频繁规则
	itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.5)
	print('频繁项集：', itemsets)
	print('关联规则：', rules)
	end = time.time()
	print("用时：", end-start)

In [6]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [7]:
# 采用mlxtend.frequent_patterns工具包
def rule2():
	from mlxtend.frequent_patterns import apriori
	from mlxtend.frequent_patterns import association_rules
	pd.options.display.max_columns=100
	start = time.time()
	hot_encoded_df=data.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')
	hot_encoded_df = hot_encoded_df.applymap(encode_units)
	frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
	rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
	print("频繁项集：", frequent_itemsets)
	print("关联规则：", rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5) ])
	#print(rules['confidence'])
	end = time.time()
	print("用时：", end-start)

In [8]:
rule1()
print('-'*100)
rule2()

频繁项集： {1: {('scandinavian',): 275, ('hot chocolate',): 552, ('cookies',): 515, ('muffin',): 364, ('pastry',): 815, ('coffee',): 4528, ('bread',): 3096, ('medialuna',): 585, ('tea',): 1350, ('farm house',): 371, ('juice',): 365, ('soup',): 326, ('cake',): 983, ('sandwich',): 680, ('alfajores',): 344, ('brownie',): 379, ('truffles',): 192, ('toast',): 318, ('scone',): 327}, 2: {('bread', 'coffee'): 852, ('bread', 'pastry'): 276, ('coffee', 'pastry'): 450, ('coffee', 'medialuna'): 333, ('coffee', 'tea'): 472, ('bread', 'tea'): 266, ('coffee', 'juice'): 195, ('coffee', 'hot chocolate'): 280, ('coffee', 'cookies'): 267, ('cake', 'coffee'): 518, ('cake', 'tea'): 225, ('bread', 'cake'): 221, ('coffee', 'sandwich'): 362, ('coffee', 'toast'): 224}}
关联规则： [{pastry} -> {coffee}, {medialuna} -> {coffee}, {juice} -> {coffee}, {hot chocolate} -> {coffee}, {cookies} -> {coffee}, {cake} -> {coffee}, {sandwich} -> {coffee}, {toast} -> {coffee}]
用时： 0.09279513359069824
----------------------------------

In [10]:
## MovieActors

In [11]:
## 下载某个演员/导演的电影数据集

In [73]:
from lxml import etree
import time
from selenium import webdriver
import pandas as pd
from efficient_apriori import apriori
import csv

这里我们需要使用ChromeDrvier来做模拟

Step1，打开谷歌浏览器， 在地址栏输入 chrome://version/  查看版本信息

Step2，ChromeDriver版本下载地址：http://chromedriver.storage.googleapis.com/index.html

Step3，放到Python\Lib\site-packages相应路径

In [74]:
driver = webdriver.Chrome('/Applications/chromedriver')

In [80]:
# 设置想要下载的导演 数据集
director = u'徐峥'
base_url = 'https://movie.douban.com/subject_search?search_text='+director+'&cat=1002&start='

movie_actors = {}
# 下载指定页面的数据
def download(request_url):
	driver.get(request_url)
	time.sleep(1)
	html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
	html = etree.HTML(html)
	# 设置电影名称，导演演员 的XPATH
	movie_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']")
	name_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='meta abstract_2']")
	# 获取返回的数据个数
	num = len(movie_lists)
	
	if num > 15: #第一页会有16条数据
		# 默认第一个不是，所以需要去掉
		movie_lists = movie_lists[1:]
		name_lists = name_lists[1:]
	for (movie, name_list) in zip(movie_lists, name_lists):
		# 会存在数据为空的情况
		if name_list.text is None: 
			continue
		# 显示下演员名称
		names = name_list.text.split('/')
		movie_actors[movie.text] = name_list.text.replace(" ", "")
		print(name_list.text.replace(" ", ""))
	print('OK') # 代表这页数据下载成功
	if num >= 15:
		# 继续下一页
		return True
	else:
		# 没有下一页
		return False

# 开始的ID为0，每页增加15
start = 0
while start<10000: #最多抽取1万部电影
	request_url = base_url + str(start)
	# 下载数据，并返回是否有下一页
	flag = download(request_url)
	if flag:
		start = start + 15
	else:
		break

# 将字典类型转化为DataFrame
movie_actors = pd.DataFrame(movie_actors, index=[0])
#print(movie_actors)
# DataFrame 行列转换
movie_actors = pd.DataFrame(movie_actors.values.T, index=movie_actors.columns, columns=movie_actors.index)
movie_actors.index.name = 'title'
movie_actors.set_axis(['actors'], axis='columns', inplace=True)
movie_actors.to_csv('~/data/Issue_004/movie_actors.csv')
print('finished')

文牧野/徐峥/王传君/周一围/谭卓/章宇/杨新鸣/王佳佳/王砚辉
宁浩/徐峥/陈思诚/闫非/彭大魔/邓超/俞白眉/葛优/黄渤/范伟/沈腾/张占义/王宝强
陈凯歌/张一白/管虎/薛晓路/徐峥/宁浩/文牧野/黄渤/张译/韩昊霖/杜江/葛优/刘昊然/宋佳/王千源
宁浩/郭涛/刘桦/连晋/黄渤/徐峥/优恵/罗兰/王迅
黄渤/舒淇/王宝强/张艺兴/于和伟/王迅/李勤勤/李又麟
贾樟柯/赵涛/廖凡/徐峥/梁嘉艳/刁亦男/张一白/丁嘉丽/张译
吴京/章子怡/徐峥/沈腾/韩昊霖/黄轩/宋佳/欧豪
宁浩/黄渤/沈腾/汤姆·派福瑞/马修·莫里森/徐峥/于和伟/雷佳音/刘桦
宁浩/徐峥/黄渤/余男/多布杰/王双宝/巴多/杨新鸣/郭虹
宁浩/黄渤/徐峥/袁泉/周冬雨/陶慧/岳小军/沈腾/张俪
徐峥/王宝强/黄渤/陶虹/谢楠/范冰冰
宁浩/黄渤/戎祥/九孔/徐峥/王双宝/巴多/董立范/高捷
徐峥/黄梅莹/袁泉/郭京飞/黄景瑜/贾冰/高以翔/沈腾
任静/严敏/黄渤/孙红雷/黄磊/罗志祥/张艺兴/王迅/陈乔恩/郭涛
徐峥/包贝尔/杜鹃/葛民辉/李灿森/潘虹/赵有亮/朱媛媛
OK
苏伦/雷佳音/佟丽娅/张衣/于和伟/王正佳/陶虹/李念/李光洁
陈正道/徐峥/莫文蔚/胡静/吕中/王耀庆/杨凯迪
叶伟民/徐峥/王宝强/李曼/李小璐/左小青/张歆艺/黄小蕾/马健
谭晓虹/李诞/王思文/王建国/张雨绮/罗永浩/庞博/呼兰/杨蒙恩
谭晓虹/于谦/李诞/吴昕/池子/马东/王建国/程璐/庞博
彭浩翔/杨千嬅/余文乐/杨幂/徐峥/陈逸宁/林兆霞/谷德昭/詹瑞文
任鹏远/徐峥/王丽坤/王砚辉/段博文/任达华/于和伟/朱珠/赵达
张建亚/徐峥/刘仪伟/宁静/白冰/伊能静/车永莉/沈星/宋佳
韩三平/黄建新/刘烨/冯远征/张嘉益/陈坤/马少骅/李沁/周润发/赵本山
嵇天毅/于和伟/张丹峰/侯梦莎/傅浤鸣/徐洪浩/吴秀波/徐峥/侯勇
杨庆/徐峥/李小璐/乔任梁/杨青/张嘉益/赵英俊
金依萌/范冰冰/李治廷/吴佩慈/蒋劲夫/丹尼尔·海尼/刘维/倪虹洁/黎明
张建亚/林嘉欣/邓超/黄渤/范伟/古巨基/黄磊/苏有朋/佟大为
黄渤/邓超/姚晨/陈凯歌/成龙/胡歌/王宝强/王源
刘仪伟/黄磊/谢娜/孙天宇/关悦/王晶/徐峥/佟大为
OK
范小天/叶崇铭/梦继/徐峥/陶虹/孙兴/李立群/陈红/翁虹/屈中恒/钮承泽
郑军/徐峥/陈好/

In [82]:
# 分析MovieLens 电影分类中的频繁项集和关联规则

In [83]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [84]:
# 数据加载
movies = pd.read_csv('~/data/Issue_004/movie_actors.csv')
#print(movies.head())

In [85]:
# 将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
movies_hot_encoded = movies.drop('actors',1).join(movies.actors.str.get_dummies('/'))
pd.options.display.max_columns=100
print(movies_hot_encoded.head())

            title  GangZhao  LukeZhiGangLiu  PengZhenZhong  YuanFang  一纳  丁嘉丽  \
0   我不是药神‎ (2018)         0               0              0         0   0    0   
1  我和我的家乡‎ (2020)         0               0              0         0   0    0   
2  我和我的祖国‎ (2019)         0               0              0         0   0    0   
3   疯狂的石头‎ (2006)         0               0              0         0   0    0   
4    一出好戏‎ (2018)         0               0              0         0   0    0   

   丁志城  丁志诚  丁黑  万弘杰  万茜  严典雅  严敏  严晓频  中孝介  丹尼尔·海尼  乔任梁  九孔  于和伟  于波  于荣光  \
0    0    0   0    0   0    0   0    0    0       0    0   0    0   0    0   
1    0    0   0    0   0    0   0    0    0       0    0   0    0   0    0   
2    0    0   0    0   0    0   0    0    0       0    0   0    0   0    0   
3    0    0   0    0   0    0   0    0    0       0    0   0    0   0    0   
4    0    0   0    0   0    0   0    0    0       0    0   0    1   0    0   

   于谦  仁龙  付连智  任达华  任静  任鹏远  伊一  伊春德  伊能静  

/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/ipykernel_90586/549538654.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies_hot_encoded = movies.drop('actors',1).join(movies.actors.str.get_dummies('/'))


In [86]:
# 将movieId, title设置为index
movies_hot_encoded.set_index(['title'],inplace=True)
#print(movies_hot_encoded.head())

In [87]:
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.05)

In [88]:
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
print(itemsets)

-------------------- 频繁项集 --------------------
     support      itemsets
2   0.743363          (徐峥)
6   0.150442          (黄渤)
10  0.097345      (黄渤, 徐峥)
1   0.079646          (宁浩)
7   0.070796      (宁浩, 徐峥)
8   0.070796      (黄渤, 宁浩)
11  0.070796  (黄渤, 宁浩, 徐峥)
3   0.061947          (沈腾)
0   0.053097         (于和伟)
4   0.053097         (王宝强)
5   0.053097          (陶虹)
9   0.053097      (沈腾, 徐峥)


In [89]:
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)

In [90]:
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False) 
#rules.to_csv('./rules.csv')

In [91]:
print('-'*20, '关联规则', '-'*20)
print(rules)

-------------------- 关联规则 --------------------
  antecedents consequents  antecedent support  consequent support   support  \
2    (黄渤, 徐峥)        (宁浩)            0.097345            0.079646  0.070796   
5        (宁浩)    (黄渤, 徐峥)            0.079646            0.097345  0.070796   
3    (宁浩, 徐峥)        (黄渤)            0.070796            0.150442  0.070796   
4        (黄渤)    (宁浩, 徐峥)            0.150442            0.070796  0.070796   
0        (黄渤)        (宁浩)            0.150442            0.079646  0.070796   
1        (宁浩)        (黄渤)            0.079646            0.150442  0.070796   

   confidence      lift  leverage  conviction  
2    0.727273  9.131313  0.063043    3.374631  
5    0.888889  9.131313  0.063043    8.123894  
3    1.000000  6.647059  0.060146         inf  
4    0.470588  6.647059  0.060146    1.755162  
0    0.470588  5.908497  0.058814    1.738446  
1    0.888889  5.908497  0.058814    7.646018  


In [94]:
### 分析MovieLens 电影分类中的频繁项集和关联规则

In [95]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [98]:
# 数据加载
movies = pd.read_csv('~/data/issue_004/movies.csv')
#print(movies.head())

In [99]:
# 将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
print(movies['genres'])
movies_hot_encoded = movies.drop('genres',1).join(movies.genres.str.get_dummies(sep='|'))
print(movies_hot_encoded)

pd.options.display.max_columns=100
print(movies_hot_encoded.head())

# 将movieId, title设置为index
movies_hot_encoded.set_index(['movieId','title'],inplace=True)
#print(movies_hot_encoded.head())
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.02)
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
print(itemsets)
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False) 
#rules.to_csv('./rules.csv')
print('-'*20, '关联规则', '-'*20)
print(rules)


0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
                            ...                     
27273                                         Comedy
27274                                         Comedy
27275                                      Adventure
27276                             (no genres listed)
27277                       Adventure|Fantasy|Horror
Name: genres, Length: 27278, dtype: object
       movieId                               title  (no genres listed)  \
0            1                    Toy Story (1995)                   0   
1            2                      Jumanji (1995)                   0   
2            3             Grumpier Old Men (1995)                   0   
3            4            Waiting to Exhale (1995)                   0   
4   

/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/ipykernel_90586/931687501.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies_hot_encoded = movies.drop('genres',1).join(movies.genres.str.get_dummies(sep='|'))


-------------------- 频繁项集 --------------------
     support                  itemsets
7   0.489185                   (Drama)
4   0.306987                  (Comedy)
14  0.153164                (Thriller)
12  0.151294                 (Romance)
0   0.129042                  (Action)
5   0.107743                   (Crime)
9   0.095718                  (Horror)
31  0.094325          (Romance, Drama)
26  0.093335           (Drama, Comedy)
6   0.090586             (Documentary)
1   0.085380               (Adventure)
27  0.069470         (Romance, Comedy)
32  0.068480         (Thriller, Drama)
13  0.063898                  (Sci-Fi)
28  0.062761            (Drama, Crime)
11  0.055503                 (Mystery)
8   0.051763                 (Fantasy)
29  0.045165         (Thriller, Crime)
20  0.044101           (Drama, Action)
15  0.043772                     (War)
3   0.041755                (Children)
22  0.040655        (Thriller, Action)
34  0.039336        (Thriller, Horror)
10  0.037979     